# Randomized Retail GL Generator
_Procured by Analytical Ants LLC_

This repository hosts a Python script that generates a randomized, yet logical, general ledger dataset tailored for retail businesses. It's a perfect tool for financial modeling training, educational purposes, and for anyone who needs a quick mock-up of a general ledger without diving deep into manual data entry.

For more information, please see full README: https://github.com/AnalyticalAnts/Randomized-Retail-GL-Generator/blob/main/README.md

In [10]:
import random
import csv
from datetime import datetime, timedelta

# Set seed for replicable results
random.seed(12345)

# Define accounts and their descriptions
accounts = {
    101: "Cash",
    102: "Accounts Receivable",
    103: "Inventory",
    201: "Accounts Payable",
    202: "Loan Payable",
    301: "Owner's Equity",
    401: "Sales Revenue",
    501: "Cost of Goods Sold",
    502: "Rent Expense",
    503: "Salary Expense"
}

# Start date for transactions two years ago from now
start_date = datetime.now() - timedelta(days=730)

# Initialize the running totals for each account
account_totals = {account: 0 for account in accounts}

def update_totals(entries):
    for entry in entries:
        if len(entry) == 6:  # Ensure we don't process description lines
            account = entry[2]
            dr = entry[4]
            cr = entry[5]
            # Asset and Expense accounts
            if account in [101, 102, 103, 501, 502, 503]:
                account_totals[account] += dr - cr
            # Liability, Owner's Equity, and Revenue accounts
            elif account in [201, 202, 301, 401]:
                account_totals[account] += cr - dr

def check_and_correct_equation(index, curr_date):
    assets = sum(account_totals[account] for account in [101, 102, 103])
    liabilities = sum(account_totals[account] for account in [201, 202])
    equity = account_totals[301]

    discrepancy = assets - (liabilities + equity)

    # If there's a discrepancy, correct by adjusting Owner's Equity
    if discrepancy != 0:
        correction_entry = (index, curr_date.strftime('%Y-%m-%d'), 301, accounts[301], 0 if discrepancy > 0 else -discrepancy, 0 if discrepancy < 0 else discrepancy)
        return [correction_entry]
    return []

def generate_ledger_entry(index, curr_date):
    entries = []
    trans_date = curr_date.strftime('%Y-%m-%d')
    
    batch_type_descriptions = {
        "owner_contribution": ["Owner's capital injection", "Owner adding funds", "Capital added by owner"],
        "sales": ["Weekly sales", "Revenue from sales", "Sales revenue recorded"],
        "purchase_inventory": ["Inventory purchase", "Bought goods for resale", "Added to inventory stock"],
        "rent_and_salaries": ["Monthly fixed costs", "Rent and salaries payment", "Operational costs for the month"],
        "loan_received": ["Taking out a loan", "Loan received from bank", "Received bank loan"],
        "purchase": ["Goods purchased", "Added to inventory", "Restocking products"],
        "sale": ["Items sold", "Sales completed", "Goods exchanged for money"],
        "expenses": ["Regular expenses", "Monthly bills paid", "Operational costs deducted"],
        "loan_activity": ["Loan activity", "Bank interactions", "Loan account adjustments"],
        "payable_payment": ["Debt payment", "Paid off a creditor", "Reduced payable amount"]
    }

    batch_type = random.choice(list(batch_type_descriptions.keys()))

    # Initial Capital Injection
    if index == 1:
        capital = round(random.uniform(50000, 70000), 2)
        entries.append((index, trans_date, 101, accounts[101], capital, 0))  # Debit cash
        entries.append((index, trans_date, 301, accounts[301], 0, capital))  # Credit Owner's Equity

    else:
        batch_type = random.choice(["purchase", "sale", "expenses", "loan_activity", "payable_payment"])

        # 1. Buying inventory
        if batch_type == "purchase":
            amount = round(random.uniform(500, 5000), 2)
            entries.append((index, trans_date, 103, accounts[103], amount, 0))  # Debit inventory
            entries.append((index, trans_date, 201, accounts[201], 0, amount))  # Credit accounts payable

        # 2. Making sales
        elif batch_type == "sale":
            sales_amount = round(random.uniform(2000, 10000), 2)  # Sales amount slightly higher than inventory cost for profit
            cogs = sales_amount * 0.70  # Assuming a profit margin of 30%
            # Ensure there's enough inventory to sell
            if account_totals[103] >= cogs:
                entries.append((index, trans_date, 401, accounts[401], 0, sales_amount))  # Credit sales
                entries.append((index, trans_date, 501, accounts[501], cogs, 0))  # Debit COGS
                entries.append((index, trans_date, 103, accounts[103], 0, cogs))  # Credit Inventory
                payment_type = random.choice(["cash", "credit"])
                if payment_type == "cash":
                    entries.append((index, trans_date, 101, accounts[101], sales_amount, 0))  # Debit cash
                else:
                    entries.append((index, trans_date, 102, accounts[102], sales_amount, 0))  # Debit accounts receivable

        # 3. Paying expenses
        elif batch_type == "expenses":
            rent = round(random.uniform(900, 2500), 2)
            salary = round(random.uniform(2500, 7000), 2)
            entries.append((index, trans_date, 502, accounts[502], rent, 0))    # Debit rent expense
            entries.append((index, trans_date, 503, accounts[503], salary, 0))  # Debit salary expense
            entries.append((index, trans_date, 101, accounts[101], 0, rent + salary))  # Credit cash

        # 4. Loan activities
        elif batch_type == "loan_activity":
            loan_activity = random.choice(["borrow", "repay"])
            loan_amount = round(random.uniform(5000, 10000), 2)
            if loan_activity == "borrow":
                entries.append((index, trans_date, 101, accounts[101], loan_amount, 0))  # Debit cash
                entries.append((index, trans_date, 202, accounts[202], 0, loan_amount))  # Credit loan payable
            else:
                entries.append((index, trans_date, 101, accounts[101], 0, loan_amount))  # Credit cash
                entries.append((index, trans_date, 202, accounts[202], loan_amount, 0))  # Debit loan payable


        # 5. Paying off accounts payable
        elif batch_type == "payable_payment":
            # Ensure payable amount doesn't exceed current payables
            max_payable = min(account_totals[201], round(random.uniform(500, 2000), 2))
            if max_payable > 0:
                entries.append((index, trans_date, 201, accounts[201], max_payable, 0))  # Debit accounts payable
                entries.append((index, trans_date, 101, accounts[101], 0, max_payable))  # Credit cash


    # Append description line
        desc_line = (index, "Description:", random.choice(batch_type_descriptions[batch_type]), "", "", "", "")
        entries.append(desc_line)
    
    return entries, curr_date + timedelta(days=random.randint(1, 10))

# Generate the dataset
dataset = []
index = 1
curr_date = start_date
while index <= 50:
    entries, curr_date = generate_ledger_entry(index, curr_date)
    dataset.extend(entries)
    
    update_totals(entries)
    corrections = check_and_correct_equation(index, curr_date)
    dataset.extend(corrections)
    update_totals(corrections)
    
    index += 1

# Write to a CSV file
try:
    with open('Synthetic_Retail_GL_Data.csv', 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(["BATCH", "DATE", "ACCOUNT", "ACCOUNT DESCRIPTION", "DR", "CR"])  # header row
        writer.writerows(dataset)
    print("Dataset generated and saved to Synthetic_Retail_GL_Data.csv!")
except PermissionError:
    print("Error: The file 'Synthetic_Retail_GL_Data.csv' is open or you do not have permission to write to it. Please close it and try again.")

Dataset generated and saved to Synthetic_Retail_GL_Data.csv!


In [15]:
# Assumptions for budget growth rates
growth_rates = {
    101: 0.05,   # Cash
    102: 0.03,   # Accounts Receivable
    103: 0.02,   # Inventory
    201: 0.02,   # Accounts Payable
    202: 0,      # Loan Payable (constant)
    401: 0.10,   # Sales Revenue
    501: 0.07,   # Cost of Goods Sold
    502: 0.03,   # Rent Expense
    503: 0.05    # Salary Expense
}

# Calculate yearly budgeted values
yearly_budgets = []

# Year One (2021) Budget
budget_2021 = {account: account_totals[account] for account in accounts}  # Start with the final GL values
for account, rate in growth_rates.items():
    budget_2021[account] *= (1 + rate)  # Apply growth rate
budget_2021[301] = budget_2021[101] + budget_2021[102] + budget_2021[103] - (budget_2021[201] + budget_2021[202])

# Add 2021 budgets to the yearly_budgets list
for account, value in budget_2021.items():
    yearly_budgets.append(("2021", account, accounts[account], value))

# Year Two (2022) Budget
budget_2022 = {account: budget_2021[account] for account in accounts}  # Start with 2021 budgeted values
for account, rate in growth_rates.items():
    budget_2022[account] *= (1 + rate)  # Apply growth rate
budget_2022[301] = budget_2022[101] + budget_2022[102] + budget_2022[103] - (budget_2022[201] + budget_2022[202])

# Add 2022 budgets to the yearly_budgets list
for account, value in budget_2022.items():
    yearly_budgets.append(("2022", account, accounts[account], value))

# Save the yearly budgets to a CSV file
yearly_budget_output_file = "Synthetic_Budget_Data.csv"

with open(yearly_budget_output_file, 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(["YEAR", "ACCOUNT", "ACCOUNT DESCRIPTION", "BUDGET AMOUNT"])  # header row
    writer.writerows(yearly_budgets)

yearly_budget_output_file


'Synthetic_Budget_Data.csv'

In [ ]:
## OPTIONAL BUDGETING SCRIPT. Retaining for information/design

# # Budgeting Assumptions based on the GL script
# budget_assumptions = {
#     101: "Ending Balance",  # Cash
#     102: "Ending Balance",  # Accounts Receivable
#     103: "Ending Balance",  # Inventory
#     201: "Ending Balance",  # Accounts Payable
#     202: "Ending Balance",  # Loan Payable
#     301: "Computed",        # Owner's Equity (Assets - Liabilities)
#     401: "Monthly Sales",   # Sales Revenue
#     501: "Monthly COGS",    # Cost of Goods Sold
#     502: "Monthly Rent",    # Rent Expense
#     503: "Monthly Salary"   # Salary Expense
# }

# # Approximate monthly budget figures based on potential transaction ranges in the GL script
# monthly_budget = {
#     401: random.uniform(5000, 10000),  # Sales Revenue
#     501: random.uniform(3000, 8000),   # Cost of Goods Sold
#     502: random.uniform(1000, 3000),   # Rent Expense
#     503: random.uniform(2000, 5000)    # Salary Expense
# }

# # Calculate budget for the entire period (assumed 2 years based on GL script)
# budget_period = 24  # 2 years

# # Extrapolate monthly figures for the entire period
# for account, amount in monthly_budget.items():
#     monthly_budget[account] = amount * budget_period

# # Calculate ending balances for balance sheet accounts based on actual data
# # Note: This is a simplification and in a real-world scenario, more sophisticated forecasting might be required.
# ending_balances = {
#     101: account_totals[101],  # Cash
#     102: account_totals[102],  # Accounts Receivable
#     103: account_totals[103],  # Inventory
#     201: account_totals[201],  # Accounts Payable
#     202: account_totals[202]   # Loan Payable
# }

# # Calculate Owner's Equity as Assets - Liabilities
# ending_balances[301] = sum(account_totals[account] for account in [101, 102, 103]) - sum(account_totals[account] for account in [201, 202])

# # Compile the budget data
# budget_data = {}
# for account, description in accounts.items():
#     if budget_assumptions[account] == "Ending Balance":
#         budget_data[account] = ending_balances[account]
#     elif budget_assumptions[account] == "Computed":
#         budget_data[account] = ending_balances[account]
#     else:
#         budget_data[account] = monthly_budget[account]

# # Write the budget data to a CSV file
# budget_filename = "Budget_Output.csv"
# with open(budget_filename, 'w', newline='') as csvfile:
#     writer = csv.writer(csvfile)
#     writer.writerow(["ACCOUNT", "ACCOUNT DESCRIPTION", "BUDGET AMOUNT"])  # header row
#     for account, amount in budget_data.items():
#         writer.writerow([account, accounts[account], amount])

# budget_filename
